In [1]:
import sys
# Change root directory
sys.path.append("../../src/")

In [2]:
from agent_design_pattern.agent import AgentMessage, BaseAgent
from agent_design_pattern.orchestration import ParallelAgent
from chain import CasualOllamaSingleTurnChain

In [3]:
class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [4]:
system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

chain = CasualOllamaSingleTurnChain(model="llama3.2:latest", system_prompt=system_prompt, user_prompt_template=user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)
agent1 = Agent(chain, state_change_callback=state_callback)
agent2 = Agent(chain, state_change_callback=state_callback)
parallel_agent = ParallelAgent([agent1, agent2], state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query), temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

query='' origin='ParallelAgent_3' response=None responses=[('Agent_1', "Fairest heart, that doth entwine\nMy soul in threads of tender design,\nA bond most strong, yet fragile as the air,\nThat whispers secrets, beyond mortal care.\n\nThy eyes, like sapphires shining bright and blue,\nReflect the depths of passions old and new,\nThe fiery blaze that burns within thy breast,\nDoth rival suns in fervent, ardent zest.\n\nAnd when I gaze upon thy lovely face,\nMy heart doth sing a love song to its place,\nA melody that echoes through my frame,\nA symphony of devotion, all but tame.\n\nYet, alas! for love's fierce and mystic might,\nIt knows no bounds, nor calmness of the night,\n'Tis capricious, wild, and free from mortal sway,\nLeaving us to dance upon its whimsical way.\n\nAnd still, I'll follow thee, through joys and fears,\nThrough trials, tribulations, and through all my tears,\nFor in thy love, I find a peaceful nest,\nA haven where my heart may find its rest."), ('Agent_2', "Fairest